In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy.stats import norm
pd.options.mode.chained_assignment = None

In [2]:
class OptionsPricingModel:
    """Black-Scholes-Merton model for SPY options pricing"""
    
    def __init__(self, spot_price, risk_free_rate, dividend_yield):
        self.S0 = spot_price
        self.r = risk_free_rate
        self.q = dividend_yield  # SPY dividend yield
        
    def black_scholes_price(self, K, T, sigma, option_type='call'):
        """
        Calculate Black-Scholes price for European options
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Implied volatility
        option_type: 'call' or 'put'
        """
        d1 = (np.log(self.S0 / K) + (self.r - self.q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        
        if option_type == 'call':
            price = self.S0 * np.exp(-self.q * T) * norm.cdf(d1) - K * np.exp(-self.r * T) * norm.cdf(d2)
        else:  # put
            price = K * np.exp(-self.r * T) * norm.cdf(-d2) - self.S0 * np.exp(-self.q * T) * norm.cdf(-d1)
        
        return price
    
    
    def crr_binomial_price(self, K, T, sigma, option_type='call', steps=1000, american=True):
        """
        Calculate option price using Cox-Ross-Rubinstein binomial tree
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Volatility
        option_type: 'call' or 'put'
        steps: Number of time steps in the binomial tree
        american: True for American options, False for European
        """
        dt = T / steps  # Time increment
        
        # CRR parameters
        u = np.exp(sigma * np.sqrt(dt))  # Up factor
        d = 1 / u  # Down factor
        p = (np.exp((self.r - self.q) * dt) - d) / (u - d)  # Risk-neutral probability
        discount = np.exp(-self.r * dt)  # Discount factor per step
        
        # Initialize asset prices at maturity
        ST = np.zeros(steps + 1)
        for i in range(steps + 1):
            ST[i] = self.S0 * (u ** (steps - i)) * (d ** i)
        
        # Initialize option values at maturity
        if option_type == 'call':
            option_values = np.maximum(ST - K, 0)
        else:  # put
            option_values = np.maximum(K - ST, 0)
        
        # Backward induction through the tree
        for step in range(steps - 1, -1, -1):
            for i in range(step + 1):
                # Current stock price at this node
                S = self.S0 * (u ** (step - i)) * (d ** i)
                
                # Continuation value (discounted expected value)
                continuation_value = discount * (p * option_values[i] + (1 - p) * option_values[i + 1])
                
                # Early exercise value
                if option_type == 'call':
                    exercise_value = max(S - K, 0)
                else:  # put
                    exercise_value = max(K - S, 0)
                
                # For American options, take the maximum
                if american:
                    option_values[i] = max(continuation_value, exercise_value)
                else:
                    option_values[i] = continuation_value
        
        return option_values[0]

In [3]:
#Scenario Analysis Result in code/1_options_trade_scenario_analysis.ipynb
pd.read_csv('../data/Result/1_Scenario_Analysis_Result.csv')

,Spot Move(%),scenario,S_0,S_t,put_iv_0,Change_in_put_iv (%),put_iv_t,call_iv_0,Change_in_call_iv (%),call_iv_t,V_p_0,Change_in_put_price (%),V_p_t,V_c_0,Change_in_call_price (%),V_c_t,pnl_dollars,pnl_percentage,avg_pnl_for_spot_move
0,15.0,vol_collapse,543.37,624.88,0.3233,-25.0,0.2425,0.2109,-9.0,0.1920,8.475,-100.0000,0.0000,4.225,1182.8066,54.1986,10867203.99,12.82,"10,867,491.14"
1,15.0,normal_decline,543.37,624.88,0.3233,-17.0,0.2683,0.2109,-6.0,0.1983,8.475,-100.0000,0.0000,4.225,1182.8240,54.1993,10867350.59,12.82,"10,867,491.14"
2,15.0,mixed_regime,543.37,624.88,0.3233,-10.0,0.2910,0.2109,-3.0,0.2046,8.475,-100.0000,0.0000,4.225,1182.8489,54.2004,10867561.84,12.82,"10,867,491.14"
3,15.0,persistent_vol,543.37,624.88,0.3233,-4.0,0.3104,0.2109,0.0,0.2109,8.475,-99.9998,0.0000,4.225,1182.8829,54.2018,10867848.15,12.82,"10,867,491.14"
4,10.0,vol_collapse,543.37,597.71,0.3233,-18.0,0.2651,0.2109,-7.0,0.1962,8.475,-99.9985,0.0001,4.225,546.6878,27.3226,5478548.89,6.46,"5,492,550.97"
5,10.0,normal_decline,543.37,597.71,0.3233,-13.0,0.2813,0.2109,-5.0,0.2004,8.475,-99.9955,0.0004,4.225,547.4504,27.3548,5484983.78,6.47,"5,492,550.97"
6,10.0,mixed_regime,543.37,597.71,0.3233,-7.0,0.3007,0.2109,-2.0,0.2067,8.475,-99.9868,0.0011,4.225,548.7154,27.4082,5495626.18,6.48,"5,492,550.97"
7,10.0,persistent_vol,543.37,597.71,0.3233,-2.0,0.3168,0.2109,2.0,0.2152,8.475,-99.9718,0.0024,4.225,550.5506,27.4858,5511045.03,6.50,"5,492,550.97"
8,5.0,vol_collapse,543.37,570.54,0.3233,-12.0,0.2845,0.2109,-5.0,0.2004,8.475,-99.5889,0.0348,4.225,48.0280,6.2542,1250868.38,1.48,"1,296,417.13"
9,5.0,normal_decline,543.37,570.54,0.3233,-7.0,0.3007,0.2109,-3.0,0.2046,8.475,-99.3348,0.0564,4.225,51.1752,6.3872,1275375.07,1.50,"1,296,417.13"


In [4]:
# -----------------------------------------------
# IV Sensitivity for Bullish Reverse Risk Strategy
# -----------------------------------------------
# We use a CRR binomial options pricing model to see
# how the prices of:
#   - A short-dated PUT (strike 516)
#   - A short-dated CALL (strike 571)
# change as we bump their implied volatilities up and down.
#
# Context:
# - This is useful for analyzing how a bullish reverse risk
#   strategy behaves when IV shifts after a large magnitude spot move.
#
# Assumptions:
# - American-style options (american=True)
# - 7 days to maturity (T = 7/365 in years)
# - We start from an initial implied vol for each leg and
#   multiply it by different factors to stress test vega.

model = OptionsPricingModel(
    spot_price=624.88,        # Current underlying price after a big move up (+15%) / replace this with 461.86 for a big down move(-15%)
    risk_free_rate=0.0433, # Annualized risk-free rate (e.g., 4.33%)
    dividend_yield=0.0127, # Continuous dividend yield (1.27% per year)
)

# Volatility multipliers to stress IV up and down; 0.25x ... 2x of initial IV
vol_multiplier = [0.25, 0.5, 0.75, 0.85, 1, 1.15, 1.25, 1.5, 2]

# Initial implied vols for each option leg
initial_p_vol = 0.323  # Put IV
initial_c_vol = 0.211  # Call IV

# Time to maturity
t = 7 / 365

print('---------------------------- Put ----------------------------')

# Loop over each vol multiplier and reprice the PUT
for m in vol_multiplier:
    bumped_sigma = initial_p_vol * m

    print(f"Vol multiplier: {m:.2f}  |  Bumped put IV: {bumped_sigma:.4f}")
    put_price = model.crr_binomial_price(
        K=516,                # Put strike
        T=t,                  # Time to maturity in years
        sigma=bumped_sigma,   # Bumped volatility
        option_type='put',    # Put option
        steps=1000,           # Number of binomial steps (higher = more accurate)
        american=True         # American-style (early exercise allowed)
    )
    print(f"Put price: {put_price:.4f}")
    print()

print('---------------------------- Call ---------------------------')

# Loop over each vol multiplier and reprice the CALL
for m in vol_multiplier:
    bumped_sigma = initial_c_vol * m

    print(f"Vol multiplier: {m:.2f}  |  Bumped call IV: {bumped_sigma:.4f}")
    call_price = model.crr_binomial_price(
        K=571,                # Call strike
        T=t,                  # Time to maturity in years
        sigma=bumped_sigma,   # Bumped volatility
        option_type='call',   # Call option
        steps=1000,           # Number of binomial steps
        american=True         # American-style
    )
    print(f"Call price: {call_price:.4f}")
    print()  


---------------------------- Put ----------------------------
Vol multiplier: 0.25  |  Bumped put IV: 0.0808
Put price: 0.0000

Vol multiplier: 0.50  |  Bumped put IV: 0.1615
Put price: 0.0000

Vol multiplier: 0.75  |  Bumped put IV: 0.2423
Put price: 0.0000

Vol multiplier: 0.85  |  Bumped put IV: 0.2746
Put price: 0.0000

Vol multiplier: 1.00  |  Bumped put IV: 0.3230
Put price: 0.0000

Vol multiplier: 1.15  |  Bumped put IV: 0.3715
Put price: 0.0006

Vol multiplier: 1.25  |  Bumped put IV: 0.4037
Put price: 0.0024

Vol multiplier: 1.50  |  Bumped put IV: 0.4845
Put price: 0.0233

Vol multiplier: 2.00  |  Bumped put IV: 0.6460
Put price: 0.2876

---------------------------- Call ---------------------------
Vol multiplier: 0.25  |  Bumped call IV: 0.0527
Call price: 54.2018

Vol multiplier: 0.50  |  Bumped call IV: 0.1055
Call price: 54.2018

Vol multiplier: 0.75  |  Bumped call IV: 0.1583
Call price: 54.2018

Vol multiplier: 0.85  |  Bumped call IV: 0.1793
Call price: 54.2022

Vol mu

### Maximium PNL Deduction
In the scenario analysis when the stock rallies by around +15% after the initial move on April 9, the IV of the options does not significantly affect the PnL. This is because the vega of both the now-ITM call and the far-OTM put becomes very low. This behavior is consistent with our scenario-analysis results in `1_options_trade_scenario_analysis.ipynb`, where the PnL shows little variation in strong rally scenarios: the call is already deep ITM, and the put is far OTM.

Therefore, the maximum PnL for the trade structure occurs when the stock continues to rally after April 9 over the following three weeks, resulting in a PnL of approximately $10,867,000.00, which corresponds to about 12.82% of the initial put-premium collected.

A 15% rally after April 9 is plausible if trade tensions continue to ease, leading to a stronger economic outlook in the weeks that follow.

---
### Minimum PnL Deduction

In the scenario analysis, when the stock **declines** by around **–15%** after the initial move on April 9, the IV of the options does not significantly affect the PnL. This is because the vega of both the now-OTM call and the deep-ITM put becomes very low. This behavior is consistent with our scenario-analysis results in `1_options_trade_scenario_analysis.ipynb`, where the PnL shows limited sensitivity to IV changes in large selloff scenarios: the put is already deep ITM, and the call is far OTM.

Therefore, the minimum PnL for the trade structure occurs when the stock continues to decline after April 9 over the following three weeks, resulting in a PnL of approximately **–$5,428,000**, which corresponds to about **–6.40%** of the initial put premium collected.

A –15% move after April 9 is plausible if trade-tariff rollback policies are reversed or if another black-swan event occurs, triggering a sharp market selloff.

---
### Evidence using Historical Data
A move of this size on the spot price is historically possible, as seen in our backtesting using the big rally days in `2.2_trade_backtesting_and_analysis.ipynb` , where the three-week return after a major rally has ranged from a pullback of -20% to a further rally of +15%.
